# Disclaimer & Copyright

Copyright 2024 : Forusone - shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini - Image analysis and software development with ER-Diagram

* This notebook explains how to use Gemini to understand images in multimodality features of Gemini.  
* Multimodality is one of the core capabilities in the latest foundation models and Gemini models are fundamentally built on this multimodality.
* In this demo, we can understand how to leverage multimodality to improve the productivity of software development.

***https://cloud.google.com/vertex-ai/generative-ai/docs/learn/overview***

## Install aiplatform library
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [9]:
%pip install --upgrade --quiet google-cloud-aiplatform

In [10]:
from IPython.display import display, Markdown

## Authentication to access to the GCP

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [11]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Mount to the google drive to access the .ipynb files in the repository.

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [13]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.

In [15]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

In [31]:
import base64

def get_encoded_content(location_type, location, mime_type ):
  """
  Get the encoded content object.

  location_type :
    The type of the location. ( local or GCS )
  location :
    The file location of the content.
  mime_type :
    The mime type of the content.

  Returns:
    The encoded content object.

  """

  content_obj = None

  if location_type == "local":
    with open(location, 'rb') as f:
      raw_obj = base64.b64encode(f.read()).decode('utf-8')
      content_obj = Part.from_data(data=base64.b64decode(raw_obj), mime_type=mime_type)

  elif location_type == "GCS":
        content_obj = Part.from_uri(location, mime_type=mime_type)
  else:
    raise ValueError("Invalid location type.")

  return content_obj


### Multimodality basic code

In [24]:

def generate(content_obj, query:str):
    """
    Generate a response from the model.

    content_obj :
      encoded object being analyzed in the process
    query :
      query to be sent to the model

    Returns:
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [content_obj, query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

In [32]:
import time

# When using Google Cloud Storage for the location type.

# location_type = "GCS"
# mime_type = "image/jpeg"
# repository_root = "gs://bucket_name"
# file_path = "/ERD.jpeg"
# location = repository_root + file_path

# When using local storage for the file location.
location_type = "local"
mime_type = "image/jpeg"

repository_root = "/content/drive/MyDrive/projects"
file_path = "/google_gen_ai_sample/contents/images/ERD.jpeg"
location = repository_root + file_path

content_obj = get_encoded_content(location_type, location, mime_type )

prompt = """You are an AI assistant helping software development.
Write down python code in the following instruction.

1. Refer to the ERD to write a code.
2. Retrieve the result from joining tables in the ERD.
3. Get the information customer id, delivery id, payment id by customer id.

"""

outcome = generate(content_obj, prompt)

display(Markdown(outcome))


```python
import sqlite3

# Connect to the database
conn = sqlite3.connect('pizza.db')

# Create a cursor object
cursor = conn.cursor()

# Define the SQL query
sql = """
SELECT
  c.CustomerID,
  d.DeilveryID,
  p.PaymentID
FROM Customer AS c
JOIN OrderHeader AS oh
  ON c.CustomerID = oh.CustomerID
JOIN Delivery AS d
  ON oh.OrderID = d.OrderID
JOIN Payment AS p
  ON oh.OrderID = p.OrderID;
"""

# Execute the query
cursor.execute(sql)

# Fetch all the rows
rows = cursor.fetchall()

# Print the results
for row in rows:
  print(row)

# Close the connection
conn.close()
```